## **Applied Data Science Final Project**

##### Megha Polavarapu (mp4392), Xiaoying Wang (xw2993), Iris June Chang (ijc2119), Mengyan Li (ml4779)

### Data Acquisition

In [3]:
import os
import pandas as pd
import requests
import json
import time
import dateutil
import datetime
from dateutil.relativedelta import relativedelta
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [90]:
# pip install fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 685.4/685.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
## 
# !unzip data.zip

#### **Ballot Data for 2020 Election**
Data Source Link: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/PQQ3KV 

Citation: 
Kuriwaki, Shiro; Reece, Mason; Baltz, Samuel; Conevska, Aleksandra; Loffredo, Joseph R.; Samarth, Taran; Mutlu, Can; Acevedo Jetter, Kevin E.; Garai, Zachary Djanogly; Murray, Kate; Hirano, Shigeo; Lewis, Jeffrey B.; Snyder, James M. Jr.; Stewart, Charles H. III, 2024, "Cast Vote Records: A Database of Ballots from the 2020 U.S. Election", https://doi.org/10.7910/DVN/PQQ3KV, Harvard Dataverse, V3

In [142]:
PATH_NAME = 'data/raw/election_data/release/'
outer_filenames = next(os.walk(PATH_NAME), (None, None, []))[1]
states = [state[6:].replace('%20',' ') for state in outer_filenames]

In [144]:
state_counties = {} 
df = pd.DataFrame()

for i, file in enumerate(outer_filenames): 
    counties = next(os.walk(PATH_NAME+file), (None, None, []))[1]
    state = states[i]
    state_counties[state] = [county[12:].replace('%20',' ') for county in counties]
    print('Loading data for ', state)
    
    for c in counties:
        file_path = PATH_NAME + file + '/' + c + '/part-0.parquet'
        d = pd.read_parquet(file_path, engine='fastparquet')
        d['state'] = state
        d['county'] = c[12:]
        df = pd.concat([df,d], axis=0)

Loading data for  MICHIGAN
Loading data for  DELAWARE
Loading data for  WISCONSIN
Loading data for  COLORADO
Loading data for  FLORIDA
Loading data for  TEXAS
Loading data for  NEW JERSEY
Loading data for  ARIZONA
Loading data for  ILLINOIS
Loading data for  WEST VIRGINIA
Loading data for  TENNESSEE
Loading data for  RHODE ISLAND
Loading data for  OREGON
Loading data for  MARYLAND
Loading data for  GEORGIA


KeyboardInterrupt: 

In [164]:
df = df[df.state != 'GEORGIA']

In [160]:
df2 = pd.DataFrame()
for i, file in enumerate(outer_filenames[14:]): 
    counties = next(os.walk(PATH_NAME+file), (None, None, []))[1]
    state = states[14+i]
    state_counties[state] = [county[12:].replace('%20',' ') for county in counties]
    print('Loading data for ', state)

    for c in counties:
        file_path = PATH_NAME + file + '/' + c + '/part-0.parquet'
        d = pd.read_parquet(file_path, engine='fastparquet')
        d['state'] = state
        d['county'] = c[12:]
        df2 = pd.concat([df2,d], axis=0)

Loading data for  GEORGIA
Loading data for  OHIO
Loading data for  UTAH
Loading data for  IOWA
Loading data for  CALIFORNIA
Loading data for  NEVADA


In [166]:
combined = pd.concat([df,df2], axis=0)

In [ ]:
combined.to_csv("data/2020_ballot.csv", index=False, encoding='utf-8-sig')

#### **Presidential Election Data 1976-2020**
Data source link: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/42MVDX

Citation:
MIT Election Data and Science Lab, 2017, "U.S. President 1976–2020", https://doi.org/10.7910/DVN/42MVDX, Harvard Dataverse, V8, UNF:6:F0opd1IRbeYI9QyVfzglUw== [fileUNF]

#### **Presidential Election Data 2020 by Precinct**
Data set link: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/NT66Z3
Citation: MIT Election Data and Science Lab, 2022, "Precinct-Level Returns 2020 by Individual State", https://doi.org/10.7910/DVN/NT66Z3, Harvard Dataverse, V6, UNF:6:aViWPnsxmDD+s1GuFrrdpA== [fileUNF]

In [157]:
PATH_NAME = 'data/raw/precinct_election_data/'

filenames = next(os.walk(PATH_NAME), (None, None, []))[2]
filenames.remove('README.md')
filenames.remove('2020-precincts-codebook.md')
filenames.remove('.Rhistory')
print(filenames)
print(len(filenames))

['2020-la-precinct-general.csv', '2020-tx-precinct-general.csv', '2020-nd-precinct-general.csv', '2020-ky-precinct-general.csv', '2020-mi-precinct-general.csv', '2020-ks-precinct-general.csv', '2020-wa-precinct-general.csv', '2020-or-precinct-general.csv', '2020-ms-precinct-general.csv', '2020-de-precinct-general.csv', '2020-ri-precinct-general.csv', '2020-oh-precinct-general.csv', '2020-ar-precinct-general.csv', '2020-al-precinct-general.csv', '2020-ut-precinct-general.csv', '2020-nj-precinct-general.csv', '2020-sd-precinct-general.csv', '2020-il-precinct-general.csv', '2020-ne-precinct-general.csv', '2020-nc-precinct-general-sorted.csv', '2020-ct-precinct-general.csv', '2020-nc-precinct-general.csv', '2020-mn-precinct-general.csv', '2020-fl-precinct-general.csv', '2020-wi-precinct-general.csv', '2020-ma-precinct-general.csv', '2020-tn-precinct-general.csv', '2020-md-precinct-general.csv', '2020-va-precinct-general.csv', '2020-ok-precinct-general.csv', '2020-ga-precinct-general.csv', 

Based on there being 52 file names, we can tell there is a duplicate (North Carolina)

In [159]:
file = 'data/raw/precinct_election_data/2020-nc-precinct-general.csv'
nc = pd.read_csv(file)
file = 'data/rawprecinct_election_data/2020-nc-precinct-general-sorted.csv'
nc_sorted = pd.read_csv(file)

print(nc.shape[0])
print(nc_sorted.shape[0])

filenames.remove('2020-nc-precinct-general-sorted.csv')

/var/folders/86/wprmb_s52rs0t4r2gfzh2df00000gn/T/ipykernel_48888/2650330106.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  nc = pd.read_csv(file)
/var/folders/86/wprmb_s52rs0t4r2gfzh2df00000gn/T/ipykernel_48888/2650330106.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  nc_sorted = pd.read_csv(file)


696804
1529852


We choose to keep the sorted data because it has more data. Based on the readMe for this dataset, contains additional information about the ballots used which could be used and therefore, this dataset will be used. 

In [274]:
# file = 'data/raw/precinct_election_data/2020-wv-precinct-general.csv'
combined = pd.DataFrame()
for file in filenames:
    print('Loading file ', file)
    d = pd.read_csv(PATH_NAME + file)
            
    # standardizing formatting in precinct column 
    d['precinct'] = d['precinct'].replace('PRECINCT ', '')
    d['votes'] = d['votes'].astype(int)
    
    # only keeping presidential election data 
    df = d.groupby(['candidate','office', 'party_detailed',	'party_simplified',	'state','state_po'], as_index=False)['votes'].sum()
    
    # re adding relevant data 
    # df = df.drop(columns = ['precinct', 'mode', 
    #        'county_name', 'county_fips', 'jurisdiction_name',
    #        'jurisdiction_fips', 'district', 'magnitude', 'dataverse',
    #        'year', 'stage', 'special', 'writein', 'state_fips', 'state_cen', 'state_ic', 'date', 'readme_check'])
    # df = df.merge(d, left_on='candidate', right_on='candidate', how='left')
    # df = df.drop_duplicates().reset_index()
    # df= df.drop(columns = ['index'])
    
    combined = pd.concat([combined,df], axis=0)

Loading file  2020-la-precinct-general.csv
Loading file  2020-tx-precinct-general.csv


/var/folders/86/wprmb_s52rs0t4r2gfzh2df00000gn/T/ipykernel_48888/3231263860.py:5: DtypeWarning: Columns (2,3,10,11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv(PATH_NAME + file)


Loading file  2020-nd-precinct-general.csv
Loading file  2020-ky-precinct-general.csv
Loading file  2020-mi-precinct-general.csv
Loading file  2020-ks-precinct-general.csv
Loading file  2020-wa-precinct-general.csv


/var/folders/86/wprmb_s52rs0t4r2gfzh2df00000gn/T/ipykernel_48888/3231263860.py:5: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv(PATH_NAME + file)


Loading file  2020-or-precinct-general.csv
Loading file  2020-ms-precinct-general.csv
Loading file  2020-de-precinct-general.csv
Loading file  2020-ri-precinct-general.csv
Loading file  2020-oh-precinct-general.csv


...

/var/folders/86/wprmb_s52rs0t4r2gfzh2df00000gn/T/ipykernel_48888/3231263860.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv(PATH_NAME + file)


Loading file  2020-pa-precinct-general.csv
Loading file  2020-mo-precinct-general.csv
Loading file  2020-az-precinct-general.csv
Loading file  2020-ny-precinct-general.csv


/var/folders/86/wprmb_s52rs0t4r2gfzh2df00000gn/T/ipykernel_48888/3231263860.py:5: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv(PATH_NAME + file)


Loading file  2020-wv-precinct-general.csv
Loading file  2020-hi-precinct-general.csv
Loading file  2020-mt-precinct-general.csv
Loading file  2020-ca-precinct-general.csv
Loading file  2020-nv-precinct-general.csv
Loading file  2020-wy-precinct-general.csv


/var/folders/86/wprmb_s52rs0t4r2gfzh2df00000gn/T/ipykernel_48888/3231263860.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv(PATH_NAME + file)


In [278]:
combined.dtypes

candidate           object
office              object
party_detailed      object
party_simplified    object
state               object
state_po            object
votes                int64
dtype: object

In [300]:
combined.head()
print([combined[col_name].unique() for col_name in combined.columns if col_name == 'state'])

[array(['LOUISIANA', 'TEXAS', 'NORTH DAKOTA', 'KENTUCKY', 'MICHIGAN',
       'KANSAS', 'WASHINGTON', 'OREGON', 'MISSISSIPPI', 'DELAWARE',
       'RHODE ISLAND', 'OHIO', 'ARKANSAS', 'ALABAMA', 'UTAH',
       'NEW JERSEY', 'SOUTH DAKOTA', 'ILLINOIS', 'NEBRASKA',
       'CONNECTICUT', 'NORTH CAROLINA', 'MINNESOTA', 'FLORIDA',
       'WISCONSIN', 'MASSACHUSETTS', 'TENNESSEE', 'MARYLAND', 'VIRGINIA',
       'OKLAHOMA', 'GEORGIA', 'COLORADO', 'DISTRICT OF COLUMBIA', 'IOWA',
       'NEW HAMPSHIRE', 'ALASKA', 'SOUTH CAROLINA', 'MAINE', 'NEW MEXICO',
       'IDAHO', 'VERMONT', 'INDIANA', 'PENNSYLVANIA', 'MISSOURI',
       'ARIZONA', 'NEW YORK', 'WEST VIRGINIA', 'HAWAII', 'MONTANA',
       'CALIFORNIA', 'NEVADA', 'WYOMING'], dtype=object)]


In [332]:
combined.to_csv('data/2020_election_vote_counts.csv', index=False, encoding='utf-8-sig')

In [342]:
## Getting who won each state 
top_candidates = combined.loc[combined.groupby(['office', 'state'])['votes'].idxmax()].reset_index(drop=True)
top_candidates = top_candidates.sort_values('state')

In [344]:
top_candidates

,candidate,office,party_detailed,party_simplified,state,state_po,votes
22835,MARY WINDOM,COURT OF CRIMINAL APPEALS JUDGE,REPUBLICAN,REPUBLICAN,ALABAMA,AL,1541862
22537,JOELETTA MARTIN BARRENTINE,CIRCUIT COURT JUDGE,REPUBLICAN,REPUBLICAN,ALABAMA,AL,26536
19299,TONYA SMITH CHESTNUT,STATE BOARD OF EDUCATION,DEMOCRAT,DEMOCRAT,ALABAMA,AL,161192
10242,TERRI A SEWELL,US HOUSE,DEMOCRAT,DEMOCRAT,ALABAMA,AL,225742
22697,DONALD J TRUMP,US PRESIDENT,REPUBLICAN,REPUBLICAN,ALABAMA,AL,1441170
...,...,...,...,...,...,...,...
8275,DONALD J TRUMP,US PRESIDENT,REPUBLICAN,REPUBLICAN,WYOMING,WY,193559
7836,BILL HENDERSON,STATE HOUSE,REPUBLICAN,REPUBLICAN,WYOMING,WY,2497
18228,LEVI J SHINKLE,STATE HOUSE,DEMOCRAT,DEMOCRAT,WYOMING,WY,816
15575,OCEAN ANDREW,STATE HOUSE,REPUBLICAN,REPUBLICAN,WYOMING,WY,3409


In [346]:
top_candidates.to_csv('data/2020_election_vote_results.csv', index=False, encoding='utf-8-sig')

#### **NYT Web Scraping**
##### References:    
- https://github.com/susannapaoli/web-scraper-nyt/tree/main    
- https://developer.nytimes.com/get-started    
- 

In [304]:
API_KEY = 'bgsBWGVUUDgyvMaTcXZZy4yd9Ogsehqh'

In [314]:
def parse_response(response):
    data = {'headline': [],  
        'date': [], 
        'web_url': [],
        'doc_type': [],
        'lead_paragraph': [],
        'material_type': [],
        'author': [],
        'section': [],
        'subsection': [],
        'keywords': []}

    articles = response['response']['docs'] 
    for article in articles: 
        date = dateutil.parser.parse(article['pub_date']).date()
        # if date != datetime.date(2020, 1, 1):
        #     continue
        if type(article['headline']) == dict and 'main' in article['headline'].keys():
            data['date'].append(date)
            data['headline'].append(article['headline']['main']) 
            if 'section_name' in article:
                data['section'].append(article['section_name'])
            else:
                data['section'].append(None)
            if 'lead_paragraph' in article:
                data['lead_paragraph'].append(article['lead_paragraph'])
            else:
                data['lead_paragraph'].append(None)
            if 'web_url' in article:
                data['web_url'].append(article['web_url'])
            else:
                data['web_url'].append(None)
            if 'subsection_name' in article:
                data['subsection'].append(article['subsection_name'])
            else:
                data['subsection'].append(None)
            if 'byline' in article:
                data['author'].append(article['byline']['original'])
            else:
                data['author'].append(None)
            data['doc_type'].append(article['document_type'])
            if 'type_of_material' in article: 
                data['material_type'].append(article['type_of_material'])
            else:
                data['material_type'].append(None)
            keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
            data['keywords'].append(keywords)
    return pd.DataFrame(data) 

In [114]:
# Get data from January 2020 to May 2020 
for i in range(1,6): 
    date = ['2020', str(i)]
    print('Working on', date)
    base_url = 'https://api.nytimes.com/svc/archive/v1/'
    url = base_url + '/' + date[0] + '/' + date[1] + '.json?api-key=' + API_KEY
    response = requests.get(url, verify=False).json()
    time.sleep(6) 
    
    df = parse_response(response)
    csv_path = 'data/raw/nyt_headlines/' + date[0] + '-' + date[1] + '.csv'
    df.to_csv(csv_path, index=False)

Working on ['2020', '1']
Working on ['2020', '2']
Working on ['2020', '3']
Working on ['2020', '4']
Working on ['2020', '5']


In [19]:
# Get data from June 2020 to November 2020 
for i in range(6,12): 
    date = ['2020', str(i)]
    print('Working on', date)
    base_url = 'https://api.nytimes.com/svc/archive/v1/'
    url = base_url + '/' + date[0] + '/' + date[1] + '.json?api-key=' + API_KEY
    response = requests.get(url, verify=False).json()
    time.sleep(6) 
    
    df = parse_response(response)
    csv_path = 'data/raw/nyt_headlines/' + date[0] + '-' + date[1] + '.csv'
    df.to_csv(csv_path, index=False)

Working on ['2020', '6']
Working on ['2020', '7']
Working on ['2020', '8']
Working on ['2020', '9']
Working on ['2020', '10']
Working on ['2020', '11']


In [322]:
files = ['data/raw/nyt_headlines/2020-' + str(i) + '.csv' for i in range(1,12)]

df = pd.DataFrame()
for f in files:
    d = pd.read_csv(f)
    df = pd.concat([df,d], axis=0)

#export to csv
df.to_csv("data/nyt_data.csv", index=False, encoding='utf-8-sig')

In [324]:
nyt = pd.read_csv('data/nyt_data.csv')

In [326]:
nyt.describe()

,headline,date,web_url,doc_type,lead_paragraph,material_type,author,section,subsection,keywords
count,51125,51128,51128,51128,50086,49878,45225,51127,21826,51128
unique,49839,334,51127,3,43656,19,10431,51,83,32827
top,Coronavirus Briefing: What Happened Today,2020-11-03,https://www.nytimes.com/2020/10/06/world/cuomo...,article,To the Editor:,News,By The New York Times,U.S.,Politics,[]
freq,182,901,2,46933,671,36278,566,11478,5010,5114


In [ ]:
from bs4 import BeautifulSoup
import selenium 
from selenium import webdriver
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.chrome.options import Options
import time 
import pandas as pd

In [ ]:
driver = webdriver.Chrome('/usr/local/bin/chromedriver')

In [ ]:
def get_body(url):
    driver.get(url)
    article_text = ''
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    paragraph = soup.find_all('p')
    for i in paragraph:
        a = i.get_text()
        if a != 'Advertisement' and a != 'Supported by' and a != 'Send any friend a story' and a != 'As a subscriber, you have 10 gift articles to give each month. Anyone can read what you share.' and not a.startswith("By"):
            article_text += a 
            article_text += " "
    time.sleep(8)
    return article_text

In [ ]:
df = pd.read_csv("data/nyt_data.csv")

In [ ]:
## CREATE A NEW COLUMN IN THE DATAFRAME CALLED "article_doby" WHERE WE ARE GOING TO APPEND THE TEXT RETRIEVE FROM 
## THE URL. NOTE: web_url REFERS TO THE COLUMN IN THE DATASET CONTAINING THE URL OF THE ARTICLE TO PASS TO get_body.

df['article_body'] = df.apply(lambda x: get_body(x.web_url), axis=1) 

In [ ]:
driver.close()

In [ ]:
df.to_csv("FILE_PATH")

#### Fox Web Scraping

In [132]:
import requests
from bs4 import BeautifulSoup

all_urls = []
url = 'https://www.cnn.com'
data = requests.get(url).text
soup = BeautifulSoup(data, features="html.parser")
for a in soup.find_all('a', href=True):
    if a['href'] and a['href'][0] == '/' and a['href'] != '#':
        a['href'] = url + a['href']
    all_urls.append(a['href'])



In [140]:
def url_is_article(url, year='2020'):
    if url:
        if 'cnn.com/{}/'.format(year) in url and '/gallery/' not in url:
            return True
    return False

article_urls = [url for url in all_urls if url_is_article(url)]

In [142]:
print(article_urls)

[]


In [158]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import time
from urllib.parse import urljoin

def get_wayback_snapshot(base_url, date_str):
    api_url = f"https://archive.org/wayback/available?url={base_url}&timestamp={date_str}"
    response = requests.get(api_url)
    data = response.json()
    try:
        snapshot = data['archived_snapshots']['closest']['url']
        return snapshot
    except KeyError:
        return None

def scrape_titles_from_snapshot(snapshot_url, base_url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    res = requests.get(snapshot_url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')

    articles = []
    # Fox News often uses <h2> or <h4> tags with links inside
    for tag in soup.find_all(['h2', 'h4']):
        a = tag.find('a')
        if a and a.text.strip():
            title = a.text.strip()
            href = a.get('href')
            if href and not href.startswith('http'):
                href = urljoin("https://web.archive.org", href)
            articles.append((title, href))
    return articles

# Loop through all days in January 2020
start_date = datetime(2020, 1, 1)
end_date = datetime(2020, 1, 10)

base_url = "https://www.foxnews.com"
data = []

while start_date <= end_date:
    date_str = start_date.strftime('%Y%m%d')
    print(f"Scraping {date_str}...")
    
    snapshot_url = get_wayback_snapshot(base_url, date_str)
    if snapshot_url:
        try:
            articles = scrape_titles_from_snapshot(snapshot_url, base_url)
            for title, url in articles:
                data.append({
                    "title": title,
                    "date": start_date.strftime('%Y-%m-%d')
                })
        except Exception as e:
            print(f"Error on {date_str}: {e}")
    else:
        print(f"No snapshot found for {date_str}")
    
    start_date += timedelta(days=1)
    time.sleep(1)  # Be respectful to archive.org

# Create DataFrame
df = pd.DataFrame(data)
print(f"\n✅ Collected {len(df)} articles from January 2020")
print(df.head())


Scraping 20200101...
No snapshot found for 20200101
Scraping 20200102...
No snapshot found for 20200102
Scraping 20200103...
Scraping 20200104...
No snapshot found for 20200104
Scraping 20200105...
No snapshot found for 20200105
Scraping 20200106...
No snapshot found for 20200106
Scraping 20200107...
Scraping 20200108...
No snapshot found for 20200108
Scraping 20200109...
Scraping 20200110...

✅ Collected 0 articles from January 2020
Empty DataFrame
Columns: []
Index: []


In [160]:
df.head()

""


In [156]:
df['url']

0       http://web.archive.org/web/20200102000431/http...
1       http://web.archive.org/web/20200102000431/http...
2       http://web.archive.org/web/20200102000431/http...
3       http://web.archive.org/web/20200102000431/http...
4       http://web.archive.org/web/20200102000431/http...
                              ...                        
1524    https://web.archive.org/web/20200110231303/htt...
1525    https://web.archive.org/web/20200110231303/htt...
1526    https://web.archive.org/web/20200110231303/htt...
1527    https://web.archive.org/web/20200110231303/htt...
1528    https://web.archive.org/web/20200110231303/htt...
Name: url, Length: 1529, dtype: object